In [1]:
import pandas as pd
import re
import os
import string
from pprint import pprint
import unicodedata as ud

In [2]:
with open('Pic-RUS_corpus_raw.txt','r+', encoding = 'utf-8') as file:
    pic_corpus_raw = file.readlines()
with open('Sib_corpus_raw.txt','r+', encoding = 'utf-8') as file:  
    sib_corpus_raw = file.readlines()
with open('NDS_corpus_raw.txt','r+', encoding = 'utf-8') as file:  
    nds_corpus_raw = file.readlines()

In [3]:
#Предварительная предобработка + создание списка нужной структуры: 0-ой элемент - текст, 1-ый элемент - источник текста. 

def splitted_text(text):
    text_spld = []
    if '\n' in text: #сразу удаляем пустые строки с \n
        text.remove('\n')
    for string in text:
        string = string.strip() #убираем пробельные символы в начале и в конце каждой строки (\t, \n в том числе)
        new_text = string.split('\t') #Разбиваем текст по табуляции (она отделяет текст от источника) и получаем список, в котором 0 элемент - наш текст, а 1-ый - его источник
        text_spld.append(new_text)
    return text_spld

In [4]:
pic_corpus = splitted_text(pic_corpus_raw)
pprint(pic_corpus)

[['Жил-был один дяденька1.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙∙ У его жены3 ∙∙∙ скоро должно было случиться день рождения1.',
  'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙ Был2 случиться.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙ {СГЛАТЫВАНИЕ}∙∙∙∙ <НРЗБ1=> ‖ ∙∙ Вот1.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙∙ Дяденька3 очень долго мучался3,', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['не знал6 как обычно6 выбрать6 ’’ подарок1 какой получше.',
  'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙∙ О-он-н36 ∙∙ ходил по магази-инам,,,', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙ выбира-ал ==', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙ думал,', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['что бы ему купи-ить...', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙ То он хотел купить су-умку,,,', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['то он хотел купить ∙∙ часы-ы,,,', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['то манеке-ен,,,', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙ но всё у него не получалось1.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙∙ Выбрать1,', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['что-нибудь стоящее2.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['∙∙ {ЧМОКАНЬЕ} ∙∙ В

In [5]:
sib_corpus = splitted_text(sib_corpus_raw)
pprint(sib_corpus)

[['Меня зовут Ка-атя,', 'Sib_01-f.eaf'],
 ['∙∙∙ {СМЕХ} ∙∙ {СМЕХ} (<И я спала> короче.)', 'Sib_01-f.eaf'],
 ['∙∙ {СМЕХ} ∙∙ ээ вот,', 'Sib_01-f.eaf'],
 ['я из Казахста-ана,', 'Sib_01-f.eaf'],
 ['и поэтому я всё лето провела в Казахстане.', 'Sib_01-f.eaf'],
 ['∙∙∙ Полина@,', 'Sib_01-f.eaf'],
 ['молчи¡!', 'Sib_01-f.eaf'],
 ['{СМЕХ} Вот.', 'Sib_01-f.eaf'],
 ['∙∙∙ ээ ∙∙ Когда я приехала,', 'Sib_01-f.eaf'],
 ['мы с друзьями решили сходить в горы,', 'Sib_01-f.eaf'],
 ['∙∙∙∙ ээ (*В= ‖ ’’ в горы мы ходили-и,', 'Sib_01-f.eaf'],
 ['у нас-с ∙∙ место называется Черкосай,', 'Sib_01-f.eaf'],
 ['∙∙∙ туда очень далеко идти вот,', 'Sib_01-f.eaf'],
 ['∙∙ и долго надо ехать,', 'Sib_01-f.eaf'],
 ['∙∙∙ {СМЕХ} ∙∙ вот,', 'Sib_01-f.eaf'],
 ['сначала мы долго ехали на-а ∙∙ машине,', 'Sib_01-f.eaf'],
 ['∙∙ потом <кстати> ==', 'Sib_01-f.eaf'],
 ['ну,', 'Sib_01-f.eaf'],
 ['ну нас высадили,', 'Sib_01-f.eaf'],
 ['∙∙∙ ээ и мы пошли ==', 'Sib_01-f.eaf'],
 ['∙∙∙ пешком в горы мы очень долго шли,', 'Sib_01-f.eaf'],
 ['(Ч

In [6]:
nds_corpus = splitted_text(nds_corpus_raw)
pprint(nds_corpus)

[['Пото= ==', 'NDS_001-f-z.eaf'],
 ['А потом ∙∙ про зайчика сон.', 'NDS_001-f-z.eaf'],
 ['∙∙ А зайчик вот,', 'NDS_001-f-z.eaf'],
 ['∙∙∙ он был в лесу.', 'NDS_001-f-z.eaf'],
 ['∙∙∙∙ А я была у р-речки.', 'NDS_001-f-z.eaf'],
 ['∙∙ {ЧМОКАНЬЕ} ∙∙ А потом ∙∙ ког= ‖ ∙∙ вот я пошла,', 'NDS_001-f-z.eaf'],
 ['∙∙ к ‖ домой,', 'NDS_001-f-z.eaf'],
 ['∙∙ а зайчик за мной,', 'NDS_001-f-z.eaf'],
 ['бежал-бежал.', 'NDS_001-f-z.eaf'],
 ['∙∙∙∙ А потом ∙∙∙ он прибежал,', 'NDS_001-f-z.eaf'],
 ['∙∙∙ ко мне опять,', 'NDS_001-f-z.eaf'],
 ['∙∙ когда я ушла уже,', 'NDS_001-f-z.eaf'],
 ['∙∙ {ЧМОКАНЬЕ} ∙∙ я вышла,', 'NDS_001-f-z.eaf'],
 ['∙∙ к себе,', 'NDS_001-f-z.eaf'],
 ['во двор.', 'NDS_001-f-z.eaf'],
 ['∙∙ И онь= ==', 'NDS_001-f-z.eaf'],
 ['∙∙ {ЧМОКАНЬЕ} ∙∙ А зайчик как-то ∙∙ сюда прискакал,', 'NDS_001-f-z.eaf'],
 ['∙∙ а потом он ∙∙ н-не знал,', 'NDS_001-f-z.eaf'],
 ['где я.', 'NDS_001-f-z.eaf'],
 ['∙∙∙∙ А я была ∙∙ у себя дома.', 'NDS_001-f-z.eaf'],
 ['∙∙∙∙ А потом этот зайчик прискакал ко мне домой.', 'NDS

In [7]:
#Функция, которая осуществляет основную предобработку текста: убирает транскрипционные обозначения, служебную разметку.

def preprocessed_text (text):
    for st in text:
        st[0] = st[0].translate(str.maketrans('ABCEHKMOPTXaceoxy','АВСЕНКМОРТХасеоху'))#заменяем латинские буквы на кириллические (и ле-ес-c. NDS_025, əво-oт1,	Pic-RUS_09)
        st[0] = re.sub(r'([Ёа-яА-Яё])-\1\1', r'\1', st[0]) #сказал-лл6,	Pic-RUS_03-m_Pr-R.eaf --> сказал6 (Что6 же-ее мне ей подарить1?».	Pic-RUS_03-m_Pr-T.eaf, пошёл-л3 в-ввə  ∙∙ детский мир3,	Pic-RUS_08-f_Pr-T.eaf)
        
        if len(re.findall(r'(\b[иИ]з-за\b)', st[0])) != 0: #Находим строки, где есть "из-за"
            words = st[0].split() #бьем строки на слова
            for i in range(len(words)): #убираем удлиненные фонемы в словах, если это не предлог "из-за": cу-умка
                  if len(re.findall(r'(\b[иИ]з-за\b)',words[i])) == 0:
                    words[i] = re.sub(r'([Ёа-яА-Яё])-\1',r'\1', words[i])
            st[0] = ' '.join(words)    
        #В строках, где нет "из-за", просто убираем удлиненные фонемы без каких-либо проверок: 
        else:
            st[0] = re.sub(r'([Ёа-яА-Яё])-\1',r'\1',st[0], flags = re.I)
        st[0] = re.sub(r'([Ёа-яА-Яё])=\s‖?\s?=\1',r'\1', st[0], flags = re.I) #как= ‖ =кая ---> какая, #и она попала в бо-ольни-иц= =цу,	NDS_013-m-z.eaf
        st[0] = re.sub(r'(?<=[Ёа-яА-Яё])=\s=(?=[Ёа-яА-Яё])', '', st[0], flags = re.I)  #по-о= =том м= =ы’ ∙∙∙ с-с ==	NDS_015-m-z.eaf ---> потом мы
       
        #Убираем остальные транскрипционные обозначения:
        st[0] = re.sub(r'([Əǝ~¿+|~—→↓‖∙’«ə»>=,@\*\(\)<0\[\]¡])|(\{\s*[А-ЯЁ]+\S*\})|(НРЗБ)|(#[А-ЯЁ]+#)|(\xa0)','',st[0]) 
       
        #Отдельно обрабатываем случаи типа ужас4н: 
        st[0] = re.sub(r'(?<=[Ёа-яА-Яё])\d{1,3}[Ёа-яА-Яё]\b','', st[0], flags = re.I) 
        #Теперь можем убрать цифры во всех остальных случаях:
        st[0] = re.sub(r'[0-9]+','', st[0])
        
        st[0] = re.sub(r'(Ё|ё)-о', r'\1', st[0],flags = re.I) #пошё-ол
        st[0] = re.sub(r'(Я|я)-а', r'\1', st[0],flags = re.I) #рождения-а, Я-а, я-ащерицу
        st[0] = re.sub(r'(Ю|ю)-у', r'\1', st[0],flags = re.I) #ночью-у
        st[0] = re.sub(r'(Ё|ё)-е', r'\1', st[0],flags = re.I) #Всё-е
        st[0] = re.sub(r'(Й|й)-(е)', r'\2', st[0],flags = re.I) #Й-ели  Ой ели мы', 'Sib_13-f.eaf\n'
        st[0] = re.sub(r'(Й|й)-(ё)', r'\2', st[0],flags = re.I) #й-ёжика
        st[0] = re.sub(r'(Й|й)-(я)', r'\2', st[0],flags = re.I) #й-яркая
        
        #Убираем лишние точки, но оставляем многоточие и точку-маркер конца предложения
        st[0] = re.sub(r'\.+(?=\s[Ёа-яА-Яё])','',st[0]) 
   
        #Убираем ударение:
        nkfd_form = ud.normalize('NFKD', st[0])
        st[0] = u"".join([c for c in nkfd_form if ud.name(c)!='COMBINING ACUTE ACCENT'])     
    
        #Убираем лишние пробелы у строк слева и справа и внутри между словами: 
        st[0] = st[0].strip()
        st[1] = st[1].strip()
        st[0] = re.sub(' +', ' ', st[0]) 
    return text         

In [8]:
pic_corpus= preprocessed_text(pic_corpus)
pprint(pic_corpus)

[['Жил-был один дяденька.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['У его жены скоро должно было случиться день рождения.',
  'Pic-RUS_01-f_Pr-R.eaf'],
 ['Был случиться.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['Вот.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['Дяденька очень долго мучался', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['не знал как обычно выбрать подарок какой получше.',
  'Pic-RUS_01-f_Pr-R.eaf'],
 ['Он ходил по магазинам', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['выбирал', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['думал', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['что бы ему купить...', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['То он хотел купить сумку', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['то он хотел купить часы', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['то манекен', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['но всё у него не получалось.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['Выбрать', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['что-нибудь стоящее.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['В итоге он отчаялся', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['пришёл спросить у своих детей:', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['может быть они дадут какой-то

In [9]:
sib_corpus= preprocessed_text(sib_corpus)
pprint(sib_corpus)

[['Меня зовут Катя', 'Sib_01-f.eaf'],
 ['И я спала короче.', 'Sib_01-f.eaf'],
 ['ээ вот', 'Sib_01-f.eaf'],
 ['я из Казахстана', 'Sib_01-f.eaf'],
 ['и поэтому я всё лето провела в Казахстане.', 'Sib_01-f.eaf'],
 ['Полина', 'Sib_01-f.eaf'],
 ['молчи!', 'Sib_01-f.eaf'],
 ['Вот.', 'Sib_01-f.eaf'],
 ['ээ Когда я приехала', 'Sib_01-f.eaf'],
 ['мы с друзьями решили сходить в горы', 'Sib_01-f.eaf'],
 ['ээ В в горы мы ходили', 'Sib_01-f.eaf'],
 ['у нас место называется Черкосай', 'Sib_01-f.eaf'],
 ['туда очень далеко идти вот', 'Sib_01-f.eaf'],
 ['и долго надо ехать', 'Sib_01-f.eaf'],
 ['вот', 'Sib_01-f.eaf'],
 ['сначала мы долго ехали на машине', 'Sib_01-f.eaf'],
 ['потом кстати', 'Sib_01-f.eaf'],
 ['ну', 'Sib_01-f.eaf'],
 ['ну нас высадили', 'Sib_01-f.eaf'],
 ['ээ и мы пошли', 'Sib_01-f.eaf'],
 ['пешком в горы мы очень долго шли', 'Sib_01-f.eaf'],
 ['Часов наверно пять.', 'Sib_01-f.eaf'],
 ['Точно шли.', 'Sib_01-f.eaf'],
 ['вот', 'Sib_01-f.eaf'],
 ['там постоянно менялось давление', 'Sib_01

In [10]:
nds_corpus= preprocessed_text(nds_corpus)
pprint(nds_corpus)

[['Пото', 'NDS_001-f-z.eaf'],
 ['А потом про зайчика сон.', 'NDS_001-f-z.eaf'],
 ['А зайчик вот', 'NDS_001-f-z.eaf'],
 ['он был в лесу.', 'NDS_001-f-z.eaf'],
 ['А я была у речки.', 'NDS_001-f-z.eaf'],
 ['А потом ког вот я пошла', 'NDS_001-f-z.eaf'],
 ['к домой', 'NDS_001-f-z.eaf'],
 ['а зайчик за мной', 'NDS_001-f-z.eaf'],
 ['бежал-бежал.', 'NDS_001-f-z.eaf'],
 ['А потом он прибежал', 'NDS_001-f-z.eaf'],
 ['ко мне опять', 'NDS_001-f-z.eaf'],
 ['когда я ушла уже', 'NDS_001-f-z.eaf'],
 ['я вышла', 'NDS_001-f-z.eaf'],
 ['к себе', 'NDS_001-f-z.eaf'],
 ['во двор.', 'NDS_001-f-z.eaf'],
 ['И онь', 'NDS_001-f-z.eaf'],
 ['А зайчик как-то сюда прискакал', 'NDS_001-f-z.eaf'],
 ['а потом он не знал', 'NDS_001-f-z.eaf'],
 ['где я.', 'NDS_001-f-z.eaf'],
 ['А я была у себя дома.', 'NDS_001-f-z.eaf'],
 ['А потом этот зайчик прискакал ко мне домой.', 'NDS_001-f-z.eaf'],
 ['Он скакал-скакал', 'NDS_001-f-z.eaf'],
 ['и прискакал.', 'NDS_001-f-z.eaf'],
 ['А потом он ск этот зайчик встретил ёжика.

In [11]:
#Создаем датафрейм
df_pic = pd.DataFrame(pic_corpus)
df_sib = pd.DataFrame(sib_corpus)
df_nds = pd.DataFrame(nds_corpus)

#Даем имена колонками и меняем местами:
df_pic.columns = ['Text', 'ID']
df_sib.columns = ['Text', 'ID']
df_nds.columns = ['Text', 'ID']
df_pic = df_pic[['ID', 'Text']]
df_sib = df_sib[['ID', 'Text']]
df_nds = df_nds[['ID', 'Text']]

#Сохраняем в csv-файл:
df_pic.to_csv('Pic.csv', index=False)
df_sib.to_csv('Sib.csv', index=False)
df_nds.to_csv('NDS.csv', index=False)